In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from matplotlib import pyplot as plt 
import math
import scipy
import pickle as pkl
import re
import datetime
pd.set_option("display.max_colwidth", None)
import warnings
warnings.filterwarnings('ignore')
import os

# Define el directorio predeterminado
directorio_predeterminado = 'C:/Users/20960898940/Documents/metricas/data/raw'

# Cambiar al directorio predeterminado
os.chdir(directorio_predeterminado)

# Ahora el directorio predeterminado es el nuevo directorio de trabajo

#### Importamos los DataFrames

In [2]:
# usuarios que mandaron @test_on @test_off @reset_user INCORPORAR LOS NUEVOS INTENT PARA TESTEO QUE SE AGREGARON EN FEBRERO DEL 2024

testers=pd.read_csv('testers.csv')
#testers=testers.f0_.values

In [3]:
rule_ne='PLBWX5XYGQ2B3GP7IN8Q-nml045fna3@b.m-1669990832420'

In [4]:
# Mesage metrics (mensajes)
#mm=pd.read_csv('mensajes.csv',sep=",")

# Definimos el tamaño del chunk, por ejemplo, 100,000 filas por chunk
chunksize = 100000

# Se crea una lista vacía para almacenar los DataFrames
chunk_list_m = []

# Lee el archivo CSV en chunks
for chunk in pd.read_csv('mensajes.csv', chunksize=chunksize):
    
    # Agregamos el chunk procesado a la lista
    chunk_list_m.append(chunk)

# Combinamos en un solo DataFrame
mm = pd.concat(chunk_list_m)

In [5]:
# Modificaciones necesesarias al campo creation_time segun la fuente de origen (Bq o Athena)

mm.creation_time=pd.to_datetime(mm.creation_time)

#mm.creation_time = pd.to_datetime(str(mm.creation_time)[10:30])

mm.creation_time=mm.creation_time.dt.tz_localize(None)

#mm = mm.rename(columns = {"vars_value": "value"})

In [6]:
# Validación de fecha mínima o de rango de fecha y hora

mm1=mm[mm['creation_time']>=np.datetime64('2024-02-19 13:00:00') ]
#mm1 = mm[(mm['creation_time'] >= np.datetime64('2024-02-23 17:00:00')) & (mm['creation_time'] <= np.datetime64('2024-02-24 18:00:00'))]

In [7]:
mm1.rule_name.nunique()

5469

In [8]:
# Convierte la columna 'creation_time' de mm1 al formato de fecha y hora.
mm1.creation_time = pd.to_datetime(mm1.creation_time)

# Redondea hacia arriba la columna 'creation_time' al segundo más cercano.
mm1.creation_time = mm1.creation_time.dt.ceil('s')

# Elimina filas duplicadas basadas en las columnas especificadas.
mm1.drop_duplicates(['session_id', 'creation_time', 'msg_from', 'rule_name'], inplace=True)

# Crea una nueva columna 'usuario' que toma los primeros 20 caracteres de la columna 'session_id'.
mm1['usuario'] = mm1.session_id.str[:20]

# Filtra las filas donde el valor de 'usuario' no está en la lista de testers.
mm1 = mm1[~mm1.usuario.isin(testers)]

# Reinicia los índices del DataFrame después de realizar las operaciones anteriores y descarta los índices anteriores.
mm1.reset_index(inplace=True, drop=True)


In [9]:
mm1.usuario.nunique()

683947

In [10]:
# Definimos el tamaño del chunk, por ejemplo, 100,000 filas por chunk
chunksize = 100000

# Se crea una lista vacía para almacenar los DataFrames
chunk_list_s = []

# Lee el archivo CSV en chunks
for chunk in pd.read_csv('clicks.csv', chunksize=chunksize):
    
    # Agregamos el chunk procesado a la lista
    chunk_list_s.append(chunk)

# Combinamos en un solo DataFrame
search = pd.concat(chunk_list_s)

# Guarda el DataFrame completo como un nuevo archivo CSV (si se necesita)
#full_df.to_csv('clicks_combined.csv', index=False)


In [11]:
# search + response = clicks

# Lee el archivo CSV y carga los datos en el DataFrame 'search'.
#search=pd.read_csv('clicks.csv',sep=",")

# Elimina filas duplicadas basadas en las columnas especificadas en el DataFrame 'search'.
search.drop_duplicates(['session_id', 'ts', 'id', 'message', 'mostrado', 'response_message'], inplace=True)

#####################################################
#EN ESTE LUGAR ESTARIA BUENO PROBAR EL MISMO REDONDEO DEL SEGUNDO EN TS PARA EVITAR EL PROBLEMA QUE TUVIMOS CON LA CANTIDAD DE CARACTERAS

# Redondea hacia arriba la columna 'creation_time' al segundo más cercano.
mm1.creation_time = mm1.creation_time.dt.ceil('s')
# Convierte la columna 'ts' (timestamp) de 'search' al formato de fecha y hora.
search.ts=pd.to_datetime(search.ts)
####################################################

# Crea una nueva columna 'usuario' que toma los primeros 20 caracteres de la columna 'session_id'.
search['usuario']=search.session_id.str[:20]

# Filtra las filas donde el valor de 'usuario' no está en la lista de testers.
search=search[~search.usuario.isin(testers)]

# Filtra las filas en las que la columna 'mostrado' es igual a la columna 'response_intent_id' con la condición 'RuleBuilder:'.
# Luego, elimina filas duplicadas basadas en la columna 'id' del resultado y almacena el resultado en 'searchcl'.
searchcl=search['RuleBuilder:'+search.mostrado==search.response_intent_id].drop_duplicates('id')

# Crea una nueva columna 'fecha' en 'search' que contiene solo la parte de la fecha de la columna 'ts'.
search['fecha']=search.ts.dt.date

In [12]:
search.head()

,ts,id,session_id,message_id,message,results_intent_name,results_intent_id,parent_intent_id,parent_intent_name,model_type,results_showable,score,rule_id,mostrado,mostrado_name,response_ts,response_message,response_intent_id,usuario,fecha
0,2025-06-30 23:59:59.537,9179634e-a2ac-47e2-b46d-b3d527acb79e,XVGJJG46OJVJF58DVTXZ_2025-06-30T22:17:51.896Z,NaN,NaN,EDU02CUX07 Apertura,PLBWX5XYGQ2B3GP7IN8Q-i0acarum5u@b.m-1698427848560,NaN,NaN,NaN,False,51.3679,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-i0acarum5u@b.m-1698427848560,EDU02CUX07 Apertura,NaN,NaN,NaN,XVGJJG46OJVJF58DVTXZ,2025-06-30
1,2025-06-30 23:59:59.537,9179634e-a2ac-47e2-b46d-b3d527acb79e,XVGJJG46OJVJF58DVTXZ_2025-06-30T22:17:51.896Z,NaN,NaN,DH11CUX02 Apertura,PLBWX5XYGQ2B3GP7IN8Q-yo4xeu539q@b.m-1714490579348,NaN,NaN,NaN,True,56.2013,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-yo4xeu539q@b.m-1714490579348,DH11CUX02 Apertura,NaN,NaN,NaN,XVGJJG46OJVJF58DVTXZ,2025-06-30
2,2025-06-30 23:59:59.537,9179634e-a2ac-47e2-b46d-b3d527acb79e,XVGJJG46OJVJF58DVTXZ_2025-06-30T22:17:51.896Z,NaN,NaN,DH08CUX01 Apertura,PLBWX5XYGQ2B3GP7IN8Q-v04ms80vox@b.m-1675187395862,NaN,NaN,NaN,True,57.8971,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-v04ms80vox@b.m-1675187395862,DH08CUX01 Apertura,NaN,NaN,NaN,XVGJJG46OJVJF58DVTXZ,2025-06-30
3,2025-06-30 23:59:59.537,9179634e-a2ac-47e2-b46d-b3d527acb79e,XVGJJG46OJVJF58DVTXZ_2025-06-30T22:17:51.896Z,NaN,NaN,DH11CUX01 Bifurcador,PLBWX5XYGQ2B3GP7IN8Q-yfzfuh48nw@b.m-1714487475484,NaN,NaN,NaN,True,61.8624,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-yfzfuh48nw@b.m-1714487475484,DH11CUX01 Bifurcador,NaN,NaN,NaN,XVGJJG46OJVJF58DVTXZ,2025-06-30
4,2025-06-30 23:59:59.537,9179634e-a2ac-47e2-b46d-b3d527acb79e,XVGJJG46OJVJF58DVTXZ_2025-06-30T22:17:51.896Z,NaN,NaN,EDU02CUX09 Apertura,PLBWX5XYGQ2B3GP7IN8Q-5njk09itvx@b.m-1724679042879,NaN,NaN,NaN,True,48.1919,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-5njk09itvx@b.m-1724679042879,EDU02CUX09 Apertura,NaN,NaN,NaN,XVGJJG46OJVJF58DVTXZ,2025-06-30


In [13]:
searchcl.head()

,ts,id,session_id,message_id,message,results_intent_name,results_intent_id,parent_intent_id,parent_intent_name,model_type,results_showable,score,rule_id,mostrado,mostrado_name,response_ts,response_message,response_intent_id,usuario
282,2025-06-30 23:59:49.843,3232caa6-e07a-4db9-b080-53a76d739a69,AJPWWCLSI67JMQAWZX48_2025-06-30T23:59:49.384Z,NPQNIMIGHVGBJU740WR6,Turno,TUR00CUX02 Turnos para salud,PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257,NaN,NaN,NaN,True,41.5397,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257,TUR00CUX02 Turnos para salud,2025-06-30 23:59:59.236,"{""button"":""Para salud"",""entities"":""{}"",""originalMessage"":""A"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257,AJPWWCLSI67JMQAWZX48
650,2025-06-30 23:59:33.135,a37ac89b-9ebe-4e75-93c4-44257d506cd0,3HVD4C2PWIEZ3YELELGG_2025-06-30T23:58:13.534Z,rF2xOsDpdHznVLxKMurx,Como hago para sacar un turno en el hospital de gastroenterología gratuito q está en la av caseros,TUR00CUX02 Mensaje inicial - Turnos,PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955,NaN,NaN,NaN,True,154.7277,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955,TUR00CUX02 Mensaje inicial - Turnos,2025-06-30 23:59:47.366,"{""button"":""Turnos"",""entities"":""{}"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955,3HVD4C2PWIEZ3YELELGG
737,2025-06-30 23:59:30.040,d2d1e133-17c7-4cdb-b94e-d5a1521b9873,CEQC1ZB8NAVTYSIZANDL_2025-06-30T23:58:56.163Z,RtE60n7c7CcfYsoXttH4,Personas en situacin de calle,DH01CAT02 Apertura,PLBWX5XYGQ2B3GP7IN8Q-mbzfm3m31j@b.m-1648495710984,NaN,NaN,NaN,True,96.6549,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-mbzfm3m31j@b.m-1648495710984,DH01CAT02 Apertura,2025-06-30 23:59:45.825,"{""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-mbzfm3m31j@b.m-1648495710984"",""button"":""Línea 108"",""entities"":""{}""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-mbzfm3m31j@b.m-1648495710984,CEQC1ZB8NAVTYSIZANDL
1023,2025-06-30 23:59:12.460,afbe55c8-0c52-4376-a033-7987a38c2558,KOKJ4PSB40GX6E6UG2WI_2025-06-30T23:57:50.246Z,fYQMFYkzr5pcoLDUfdv,En dónde puedo encontrar el libro de la cripta de los Casares por PDF,CU05CUX01 Apertura,PLBWX5XYGQ2B3GP7IN8Q-4pefum0myw@b.m-1631643781450,NaN,NaN,NaN,True,85.0240,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-4pefum0myw@b.m-1631643781450,CU05CUX01 Apertura,2025-06-30 23:59:23.967,"{""button"":""Bibliotecas de BA"",""entities"":""{}"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-4pefum0myw@b.m-1631643781450""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-4pefum0myw@b.m-1631643781450,KOKJ4PSB40GX6E6UG2WI
1251,2025-06-30 23:58:57.473,9e660d80-e718-447f-9868-c16acfc65635,I57ZWGU0IID8EI1QZMWN_2025-06-30T23:58:56.955Z,E7YNXCH1A061665C00FZ,Para sacar turno,TUR00CUX02 Turnos para salud,PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257,NaN,NaN,NaN,True,82.3684,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257,TUR00CUX02 Turnos para salud,2025-06-30 23:59:39.503,"{""button"":""Para salud"",""entities"":""{}"",""originalMessage"":""A"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-3fr1tizyrq@b.m-1674735659257,I57ZWGU0IID8EI1QZMWN


In [14]:
# user-buttons (botones)
#one=pd.read_csv('botones.csv',sep=",")

# Definimos el tamaño del chunk, por ejemplo, 100,000 filas por chunk
chunksize = 100000

# Se crea una lista vacía para almacenar los DataFrames
chunk_list_b= []

# Lee el archivo CSV en chunks
for chunk in pd.read_csv('botones.csv', chunksize=chunksize):
    
    # Agregamos el chunk procesado a la lista
    chunk_list_b.append(chunk)

# Combinamos en un solo DataFrame
one = pd.concat(chunk_list_b)

In [15]:
# Procesamiento de datos en el DataFrame 'one':  ONESHOTS

# Crea una nueva columna 'usuario' en el DataFrame 'one', que toma los primeros 20 caracteres de la columna 'session_id'.
one['usuario'] = one.session_id.str[:20]

# Filtra las filas donde el valor de 'usuario' no está en la lista de testers.
one = one[~one.usuario.isin(testers)]

# Filtra las filas en el DataFrame 'one' donde 'one_shot' es True y 'type' es 'oneShot' o 'oneShotSearch'.
os = one[np.logical_and(one.one_shot == True, one.type.isin(['oneShot', 'oneShotSearch']))]

# Convierte la columna 'ts' (timestamp) de 'os' al formato de fecha y hora. HABRIA QUE REDONDEAR LSO SEGUNDOS POR LAS DUDAS TAMBIEN?
os.ts = pd.to_datetime(os.ts)

# Crea una nueva columna 'fecha' en 'os' que contiene solo la parte de la fecha de la columna 'ts'.
os['fecha'] = os.ts.dt.date


In [16]:
#importamos el csv con la lista de los intents mostrables (tambien aparece como showable)

mos=pd.read_csv('Actualizacion_Lista_Blanca.csv')
# Elimina los espacios en blanco alrededor de los nombres de intenciones en la columna 'Nombre de la intención'.
rules_mos = mos['Nombre de la intención'].str.strip().values

### Transformaciones

In [17]:
# sacamos mensajes seguidos de boti

# Reinicia los índices del DataFrame 'mm1', descartando los índices anteriores y aplicando los cambios en el lugar.
mm1.reset_index(inplace=True, drop=True)

# Crea una lista 'drop' que contiene los índices de las filas que deben eliminarse.
drop = [i if mm1.loc[i].msg_from == mm1.loc[i+1].msg_from and mm1.loc[i].session_id == mm1.loc[i+1].session_id else None for i in mm1.index[:-1]]

# Convierte la lista 'drop' en un conjunto para eliminar duplicados y luego convierte de nuevo a lista.
drop = list(set(drop))

# Elimina los valores 'None' de la lista 'drop'.
drop.remove(None)

# Elimina las filas del DataFrame 'mm1' utilizando los índices almacenados en la lista 'drop'.
mm1.drop(drop, inplace=True)

# Reinicia los índices del DataFrame 'mm1' después de eliminar las filas, descartando los índices anteriores y aplicando los cambios en el lugar.
mm1.reset_index(inplace=True, drop=True)


In [18]:
# Selecciona las filas en el DataFrame 'mm1' donde la columna 'max_score' no es nula y muestra las primeras filas.
mm1[~mm1['max_score'].isnull()].head()


,session_id,id,creation_time,msg_from,message_type,message,rule_name,topic_path,original_user_message,max_score,usuario
11,0000DSOPIF5Z85UNH51F_2025-06-26T16:45:12.833Z,I5QEKB45P2VBGF47V0RF,2025-06-27 00:34:47,user,Text,"Quiero recibir noticias , Gracias!",NaN,NaN,NaN,52.3422,0000DSOPIF5Z85UNH51F
116,0007N0RF8M2MGIC8DV26_2025-06-13T20:51:02.665Z,zQ5cFjaLz5DxETgv3tSQ,2025-06-13 21:03:06,user,Button-click,"{""button"":""No era nada de eso"",""entities"":""{}"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-y272iwgx5g@b.m-1688392737934""}",NaN,NaN,NaN,73.6895,0007N0RF8M2MGIC8DV26
191,0008GB8RKE22E73M52G1_2025-06-23T22:09:38.485Z,CMBVYHWHQKOL8W1LR61Q,2025-06-23 22:40:59,user,Text,Patentes,NaN,NaN,NaN,66.3787,0008GB8RKE22E73M52G1
197,0008GB8RKE22E73M52G1_2025-06-29T18:55:25.323Z,DILN7EM3WDJ0OT4N24E8,2025-06-29 18:55:36,user,Text,Vtv,NaN,NaN,NaN,65.9343,0008GB8RKE22E73M52G1
212,000CGUJ37OSWLQ0JWVIR_2025-06-27T21:53:01.830Z,7I0ELR123F5M7HM66WYV,2025-06-27 21:53:02,user,Text,Turno para renovar licencia,NaN,NaN,NaN,81.8440,000CGUJ37OSWLQ0JWVIR


In [19]:
#1.2 del debbug
# Análisis de respuestas por usuario en el DataFrame 'mm1':


# Copia y reinicia los índices del DataFrame 'mm1'.
#mm = mm1.copy()
#mm.reset_index(inplace=True, drop=True)

# Filtra y selecciona columnas específicas de mensajes de texto enviados por usuarios.
#mmtex1 = mm[np.logical_and(mm.msg_from == 'user', mm.message_type == 'Text')][['session_id', 'id', 'creation_time', 'msg_from', 'message_type', 'message', 'usuario']]


In [20]:
# Debug: imprimir dimensiones y verificación paso a paso

# 1. Verificar las dimensiones de los DataFrames mm y mmtex1
#print(f"Dimensiones de mm: {mm.shape}")
#print(f"Dimensiones de mmtex1: {mmtext1.shape}")



# 2. Definir el rango válido para asegurar que no se excedan los índices de 'mm'
#valid_range = mmtex1.index[mmtex1.index + 1 < mm.shape[0]]

# Imprimir el rango válido
#print(f"Valid range: {valid_range}")
#print(f"Longitud del valid_range: {len(valid_range)}")


# 3. Verificar que no haya valores nulos en las columnas de interés dentro de los índices válidos
#valid_data_mm = (
 #   ~mm.loc[valid_range + 1,'rule_name'].isnull() &
 #   ~mm.loc[valid_range + 1,'session_id'].isnull() &
 #   ~mm.loc[valid_range + 1,'msg_from'].isnull()
#)


# Imprimir información de los datos válidos
#print(f"valid_data_mm (True/False array): {valid_data_mm}")
#print(f"valid_data_mmtex1 (True/False array): {valid_data_mmtex1}")

# 4. Unir las dos máscaras para asegurar que estamos trabajando solo con datos válidos en ambos DataFrames
#valid_data = valid_data_mm & valid_data_mmtex1

# Imprimir la máscara combinada y verificar su longitud
#print(f"valid_data combinada (True/False array): {valid_data}")
#print(f"Longitud de valid_data combinada: {len(valid_data)}")

# 5. Filtrar el rango de índices basado en los datos válidos
#filtered_range = valid_range[valid_data]

# Imprimir el rango filtrado
#print(f"Filtered range: {filtered_range}")
#print(f"Longitud del filtered_range: {len(filtered_range)}")

# 6. Verificar las dimensiones de las listas que se usarán con zip
#print(f"Longitud de mm.loc[filtered_range + 1].rule_name.values: {len(mm.loc[filtered_range + 1].rule_name.values)}")
#print(f"Longitud de mmtex1.loc[filtered_range].session_id.values: {len(mmtex1.loc[filtered_range].session_id.values)}")
#print(f"Longitud de mm.loc[filtered_range + 1].session_id.values: {len(mm.loc[filtered_range + 1].session_id.values)}")
#print(f"Longitud de mm.loc[filtered_range + 1].msg_from.values: {len(mm.loc[filtered_range + 1].msg_from.values)}")

# 7. Recorrer los valores filtrados y ver los datos
#for idx, (r, su, sb, f) in enumerate(zip(
#    mm.loc[filtered_range + 1].rule_name.values,
#    mmtex1.loc[filtered_range].session_id.values,
#    mm.loc[filtered_range + 1].session_id.values,
#   mm.loc[filtered_range + 1].msg_from.values
#)):
    # Imprimir los valores y sus índices correspondientes para depuración
 #   print(f"Índice: {filtered_range[idx]}, r: {r}, su: {su}, sb: {sb}, f: {f}")


In [21]:
#1.2.1 del debbug
# Crea la columna 'rule_name' basada en ciertas condiciones.
#mmtex1['rule_name'] = [r if su == sb and f == 'bot' else None for r, su, sb, f in zip(mm.loc[mmtex1.index + 1].rule_name.values, mmtex1.session_id.values, mm.loc[mmtex1.index + 1].session_id.values, mm.loc[mmtex1.index + 1].msg_from.values)]


#### Modelo Nuevo Primera Instancia

In [22]:
# Análisis de respuestas por usuario en el DataFrame 'mm1':


# Copia y reinicia los índices del DataFrame 'mm1'.
mm = mm1.copy()
mm.reset_index(inplace=True, drop=True)

# Filtra y selecciona columnas específicas de mensajes de texto enviados por usuarios.
mmtex1 = mm[np.logical_and(mm.msg_from == 'user', mm.message_type == 'Text')][['session_id', 'id', 'creation_time', 'msg_from', 'message_type', 'message', 'usuario']]

# Elimina la última fila del DataFrame
mmtex1 = mmtex1.iloc[:-1]

# Crea la columna 'rule_name' basada en ciertas condiciones.
mmtex1['rule_name'] = [r if su == sb and f == 'bot' else None for r, su, sb, f in zip(mm.loc[mmtex1.index + 1].rule_name.values, mmtex1.session_id.values, mm.loc[mmtex1.index + 1].session_id.values, mm.loc[mmtex1.index + 1].msg_from.values)]

# Filtra y selecciona las filas relacionadas con la regla 'No entendió letra no existente en WA'.
letra1 = mmtex1[mmtex1.rule_name == 'No entendió letra no existente en WA']
letra1.rename(columns={'id': 'message_id'}, inplace=True)

# Filtra 'search' y 'os' según 'mm1.session_id.values'.
search1 = search[search.session_id.isin(mm1.session_id.values)]
os1 = os[os.session_id.isin(mm1.session_id.values)]

# Filtra las instancias iniciales que no están en 'search1' o 'os1' y realiza algunas transformaciones.
primera_instancia1 = search[~search.message_id.isin(pd.concat([search1['RuleBuilder:' + search1.mostrado == search1.response_intent_id].message_id, os1.message_id]).values)].drop_duplicates('id')
primera_instancia1.rename(columns={"results_score": "score"}, inplace=True)
ne1 = primera_instancia1.groupby('id').max()[['session_id', 'message_id', 'score']]
ne1 = ne1[ne1.score <= 5.36]

primera_instancia1 = primera_instancia1[~primera_instancia1.id.isin(ne1.index)]
os1 = os1.drop_duplicates('id')[['session_id', 'message_id']]
click1 = search1['RuleBuilder:' + search1.mostrado == search1.response_intent_id].drop_duplicates('id')[['session_id', 'message_id']]
abandonos1 = primera_instancia1[primera_instancia1.response_message.isna()][['session_id', 'message_id']]
nada1 = primera_instancia1[primera_instancia1.response_intent_id == 'RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-alfafc@gmail.com-1536777380652'][['session_id', 'message_id']]
texto1 = primera_instancia1[np.logical_and(primera_instancia1.response_intent_id != 'RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-alfafc@gmail.com-1536777380652', ~primera_instancia1.response_message.isna())][['session_id', 'message_id']]
letra1 = letra1[['session_id', 'message_id']]

# Agrega una columna 'categoria' a 'os1', 'click1', 'abandonos1', 'nada1', 'texto1', 'ne1', y 'letra1'.
os1['categoria'] = 'one'
click1['categoria'] = 'click'
abandonos1['categoria'] = 'abandono'
nada1['categoria'] = 'nada'
texto1['categoria'] = 'texto'
ne1['categoria'] = 'ne'
letra1['categoria'] = 'letra'


# Concatena y filtra 'value1primera' según 'mm1.usuario.values'.
value1primera = pd.concat([os1, click1, abandonos1, nada1, texto1, ne1, letra1])
value1primera['usuario'] = value1primera.session_id.str[:20]
value1primera = value1primera[value1primera.usuario.isin(mm1.usuario.values)]

# Realiza un análisis de respuestas por usuario y categoría.
respuestas_por_usuario = value1primera.groupby(['usuario', 'categoria'], as_index=False).count()[['usuario', 'categoria', 'message_id']].pivot_table('message_id', ['usuario'], 'categoria')
respuestas_por_usuario.fillna(0, inplace=True)
respuestas_por_usuario = respuestas_por_usuario.reset_index(drop=False).reindex(['usuario', 'one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra'], axis=1)

# Calcula porcentajes por categoría para cada usuario.
respuestas_por_usuario['porcentaje_abandono']=[respuestas_por_usuario.loc[i].abandono / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_click']=[respuestas_por_usuario.loc[i].click / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_one']=[respuestas_por_usuario.loc[i].one / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_texto']=[respuestas_por_usuario.loc[i].texto / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_nada']=[respuestas_por_usuario.loc[i].nada / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_ne']=[respuestas_por_usuario.loc[i]['ne'] / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_letra']=[respuestas_por_usuario.loc[i]['letra'] / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]


# Crea un DataFrame 'res_primera_instancia1' basado en 'respuestas_por_usuario'.
res_primera_instancia1 = respuestas_por_usuario.copy()

# Calcula promedios para diferentes categorías.
promedios1={'abandonos': round(respuestas_por_usuario['porcentaje_abandono'].mean(), 3),     
                  'click': round(respuestas_por_usuario['porcentaje_click'].mean(), 3),
                  'one': round(respuestas_por_usuario['porcentaje_one'].mean(), 3),
                  'texto': round(respuestas_por_usuario['porcentaje_texto'].mean(), 3),
                  'nada': round(respuestas_por_usuario['porcentaje_nada'].mean(), 3),
                  'letra': round(respuestas_por_usuario['porcentaje_letra'].mean(), 3),
                  'ne': round(respuestas_por_usuario['porcentaje_ne'].mean(), 3)
}



In [23]:
cant_sessions1 = len(mm)
print("cant_sessions1:", cant_sessions1)

cant_sessions1: 16273674


In [24]:
cant_oss1 = len(os1)
print("cant_oss1:", cant_oss1)

cant_oss1: 640582


In [25]:
ne2 = primera_instancia1.groupby('id').max()[['session_id', 'message_id', 'score']]
ne2 = ne1[ne1.score <= 5.36]

In [26]:
#Descarga df con las session_id de los casos que se necesiten [En primera instancia]

#categoria = {
#    'one': os1['session_id'].head(50), [(nada.session_id.creation_time('2024-09-01' between '2024-09-01'))]
#    'click': click1['session_id'].head(50),
#    'abandono': abandonos1['session_id'].head(50),
#   'nada': nada1['session_id'].head(50),
#    'texto': texto1['session_id'].head(50),
#    'ne': ne1['session_id'],
#    'letra': letra1['session_id'].head(50)
#}

# Guarda cada categoría en un archivo CSV
#for categoria, session_ids in categoria.items():
#    session_ids.to_csv(f'{categoria}_sessionn_ids.csv', index=False)

In [27]:
promedios1

{'abandonos': np.float64(0.039),
 'click': np.float64(0.139),
 'one': np.float64(0.656),
 'texto': np.float64(0.032),
 'nada': np.float64(0.04),
 'letra': np.float64(0.078),
 'ne': np.float64(0.016)}

In [28]:
# Definimos la funcion que identifica las distintas categorías

def categoria(m, t, r):
    # Esta función categoriza mensajes basándose en el contenido del mensaje (m), el tipo de mensaje (t), y el nombre de la regla (r).

    try:
        # Verifica si el tipo de mensaje es 'Button-click' y si el mensaje contiene 'Cambiar de tema'.
        if t == 'Button-click' and 'Cambiar de tema' in m:
            return 'cambiar'

        # Verifica si el tipo de mensaje es 'Button-click' y si la regla es 'Menú show buttons'.
        elif t == 'Button-click' and r == 'Menú show buttons':
            return 'otros'

        # Verifica si el tipo de mensaje es 'Button-click' y si el mensaje contiene 'No era nada de eso'.
        elif t == 'Button-click' and 'No era nada de eso' in m:
            return 'x'

        # Verifica si el tipo de mensaje es 'Button-click'.
        elif t == 'Button-click':
            return 'boton'

        # Verifica si el mensaje es 'a', 'b', 'c', o 'd' (ignorando mayúsculas y minúsculas) y si la regla es 'Infracciones * Apertura'.
        elif re.match(r'^a$|^b$|^c$|^d$', m, re.IGNORECASE) and r == 'Infracciones * Apertura':
            return 'boton'

        # Verifica si el mensaje es 'a', 'b', 'c', o 'd' (ignorando mayúsculas y minúsculas) y si la regla es 'Busca donde está permitido estacionar'.
        elif re.match(r'^a$|^b$|^c$|^d$', m, re.IGNORECASE) and r == 'Busca donde está permitido estacionar':
            return 'boton'

        elif m == '__image__' and r == 'Denuncia Vial - Validación Vehículo':
             return 'boton'
        elif re.match(r'[0-9]{7,8}', m) and r == 'Licencia prorroga  > Consultar':
             return 'boton'

        # Verifica si el mensaje es 'x' (ignorando mayúsculas y minúsculas) o 'x buscaba otra cosa'.
        elif re.match(r'(^x$)|(x?\.? ?buscaba otra cosa)', m, re.IGNORECASE):
            return 'x'

        # Si no coincide con ninguna de las condiciones anteriores, categoriza como 'texto'.
        else:
            return 'texto'

    except:
        # Maneja excepciones y retorna 'otros' en caso de error.
        return 'otros'


In [29]:
# Análisis de interacciones del usuario en el DataFrame 'mm1':

# Copia el DataFrame 'mm1' a 'mm'.
mm = mm1.copy()

# Reinicia los índices de 'mm', descartando los índices anteriores y aplicando los cambios en el lugar.
mm.reset_index(inplace=True, drop=True)

# Filtra las filas donde 'msg_from' es 'user'.
mmu = mm[mm.msg_from == 'user']

# Reinicia los índices de 'mmu', descartando los índices anteriores y aplicando los cambios en el lugar.
mmu.reset_index(inplace=True, drop=True)

# Filtra las filas originales del usuario que están en 'searchcl.message_id'.
original = mmu[mmu.id.isin(searchcl.message_id.values)] 

# Obtiene las filas siguientes (botones) y respuestas subsiguientes.
boton = mmu.loc[original.index + 1]
respuesta = mmu.loc[original.index + 2]

# Crea un DataFrame 'conv_cl' con información de la conversación, como la sesión, la hora de creación y mensajes originales.
# También incluye información sobre el intent, el primer botón, respuestas intermedias y finales.
conv_cl = pd.DataFrame(data={'session_id': original.session_id.values, 'creation_time': original.creation_time.values, 'original': original.message.values, 
                             'intent': mm.loc[mm[mm.id.isin(boton.id.values)].index + 1].rule_name.values,
                             'bot1_id': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(boton.id.values)].index + 1].id.values, original.session_id.values==mm.loc[mm[mm.id.isin(boton.id.values)].index + 1].session_id.values)],
                             'respuesta_intermedia': [m if v else None for m, v in zip(boton.message.values, original.session_id.values==boton.session_id.values)], 
                             'respuesta': [m if v else None for m, v in zip(respuesta.message.values, original.session_id.values==respuesta.session_id.values)],
                             'respuesta_type': [m if v else None for m, v in zip(respuesta.message_type.values, original.session_id.values==respuesta.session_id.values)],
                             'respuesta_rule': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].rule_name.values, original.session_id.values==mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].session_id.values)]})

mm = mm1.copy()
mm.reset_index(inplace=True, drop=True)
mmu = mm[mm.msg_from == 'user']
mmu.reset_index(inplace=True, drop=True)

# Filtra las filas originales del usuario que están en 'os.message_id'.
original = mmu[mmu.id.isin(os.message_id.values)]

# Obtiene la respuesta subsiguiente.
respuesta = mmu.loc[original.index + 1]

# Crea un DataFrame 'conv' con información de la conversación, como la sesión, la hora de creación y mensajes originales.
# También incluye información sobre el intent, el primer botón, y respuestas subsiguientes.
conv = pd.DataFrame(data={'session_id': original.session_id.values, 'creation_time': original.creation_time.values, 'original': original.message.values, 
                          'intent': mm.loc[mm[mm.id.isin(original.id.values)].index + 1].rule_name.values,
                          'bot1_id': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(original.id.values)].index + 1].id.values, original.session_id.values==mm.loc[mm[mm.id.isin(original.id.values)].index + 1].session_id.values)],
                          'respuesta': [m if v else None for m, v in zip(respuesta.message.values, original.session_id.values==respuesta.session_id.values)],
                          'respuesta_type': [m if v else None for m, v in zip(respuesta.message_type.values, original.session_id.values==respuesta.session_id.values)],
                          'respuesta_rule': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].rule_name.values, original.session_id.values==mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].session_id.values)]})


# Crea una nueva columna 'categoria' en 'conv' utilizando la función 'categoria'.
conv['categoria'] = [categoria(m, t, r) if m is not None else 'abandono' for m, t, r in zip(conv.respuesta, conv.respuesta_type, conv.intent)]

# Agrupa por 'categoria' y cuenta la cantidad de 'bot1_id' para cada categoría.
per = conv.groupby('categoria', as_index=False).count()[['categoria', 'bot1_id']]

# Calcula el porcentaje de cada categoría respecto al total.
per['per'] = per.bot1_id / per.bot1_id.sum()

# Concatena 'conv_cl' y 'conv' en un nuevo DataFrame 'usuario1', y agrega columnas adicionales.
usuario1 = pd.concat([conv_cl[['session_id', 'creation_time', 'original', 'intent', 'bot1_id', 'respuesta', 'respuesta_type', 'respuesta_rule']], conv])

# Crea una nueva columna 'categoria' en 'usuario1' utilizando la función 'categoria'.
usuario1['categoria'] = [categoria(m, t, r) if m is not None else 'abandono' for m, t, r in zip(usuario1.respuesta, usuario1.respuesta_type, usuario1.intent)]

# Crea una nueva columna 'usuario' en 'usuario1' extrayendo los primeros 20 caracteres de 'session_id'.
usuario1['usuario'] = usuario1.session_id.str[:20]

# Crea una nueva columna 'id' en 'usuario1' copiando los valores de 'bot1_id'.
usuario1['id'] = usuario1.bot1_id



ValueError: operands could not be broadcast together with shapes (144547,) (144549,) 

In [ ]:
cant_sessions2 = len(mmu)
print("cant_sessions2:", cant_sessions2)

cant_sessions2: 1625939


#### Resultados

In [ ]:
# Análisis de resultados por usuario:

resultados=[]
promedios=[]

# Itera sobre el DataFrame 'usuario1'.
for usuario in [usuario1]:

    # Agrupa por 'usuario' y 'categoria', y cuenta la cantidad de 'id' para cada categoría.
    respuestas_por_usuario=usuario.groupby(['usuario','categoria'], as_index=False).count()[['usuario','categoria', 'id']].pivot_table('id', ['usuario'], 'categoria')
    respuestas_por_usuario.fillna(0, inplace=True)

    # Reordena las columnas del DataFrame 'respuestas_por_usuario'.
    respuestas_por_usuario=respuestas_por_usuario.reset_index(drop=False).reindex(['usuario', 'abandono', 'boton', 'otros', 'texto', 'x', 'cambiar'], axis=1)

    # Calcula los porcentajes para cada categoría.
    respuestas_por_usuario['porcentaje_abandono']=[respuestas_por_usuario.loc[i].abandono / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_boton']=[respuestas_por_usuario.loc[i].boton / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_otros']=[respuestas_por_usuario.loc[i].otros / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_texto']=[respuestas_por_usuario.loc[i].texto / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_x']=[respuestas_por_usuario.loc[i].x / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_cambiar']=[respuestas_por_usuario.loc[i].cambiar / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]

    # Agrega el DataFrame 'respuestas_por_usuario' a la lista 'resultados'.
    resultados.append(respuestas_por_usuario)

    # Calcula los promedios de porcentajes para cada categoría y agrega a la lista 'promedios'.
    promedios.append({'abandonos': round(respuestas_por_usuario['porcentaje_abandono'].mean(), 3),     
                      'botones': round(respuestas_por_usuario['porcentaje_boton'].mean(), 3),
                      'otros': round(respuestas_por_usuario['porcentaje_otros'].mean(), 3),
                      'texto': round(respuestas_por_usuario['porcentaje_texto'].mean(), 3),
                      'x': round(respuestas_por_usuario['porcentaje_x'].mean(), 3),
                      'cambiar de tema': round(respuestas_por_usuario['porcentaje_cambiar'].mean(), 3)})



In [ ]:
#casos_x = respuestas_por_usuario['porcentaje_x']=[respuestas_por_usuario.loc[i]].x 
#casos_x = respuestas_por_usuario[['usuario', 'x']]
# Extrae los valores crudos de la columna 'x' y los agrega a la lista 'valores_x'.
#casos_x.extend(respuestas_por_usuario['x'].values)

# 'valores_x' ahora contiene los valores crudos de 'x' para cada usuario.
#print(casos_x)

# Guardar el DataFrame en un archivo CSV
#casos_x.to_csv('casos_x.csv', index=False)

In [ ]:
# Filtramos el DataFrame para obtener las sesiones con la columna 'x'
#sesiones_botones = respuestas_por_usuario[['usuario', 'boton']]
#sesiones_x = respuestas_por_usuario[['x']]
# Si deseas incluir solo las sesiones donde 'x' tenga valores mayores a 0
# sesiones_x = sesiones_x[sesiones_x['x'] > 0]

# Guardar el DataFrame en un archivo CSV
#sesiones_x.to_csv('sesiones_x.csv', index=False)


In [ ]:
pd.DataFrame(promedios, index=['nuevo-con-oss', 'nuevo-sin-oss'])[['abandonos', 'botones', 'texto', 'x', 'cambiar de tema', 'otros']]

,abandonos,botones,texto,x,cambiar de tema,otros
nuevo-con-oss,0.109,0.752,0.086,0.025,0.029,0.0
nuevo-sin-oss,0.109,0.752,0.086,0.025,0.029,0.0


In [ ]:
# Crea un nuevo diccionario 'con_oss1' multiplicando cada valor en 'promedios1' por 100.
con_oss1 = {k: v * 100 for k, v in promedios1.items()}

# Crea un nuevo diccionario 'con_oss2' multiplicando los valores correspondientes en 'promedios1' por las sumas ponderadas de 'click' y 'one' en 'promedios'.
con_oss2 = {k: (promedios1['click'] + promedios1['one']) * v * 100 for k, v in promedios[0].items()}


In [ ]:
con_oss1

{'abandonos': np.float64(2.9000000000000004),
 'click': np.float64(13.700000000000001),
 'one': np.float64(68.10000000000001),
 'texto': np.float64(2.4),
 'nada': np.float64(3.8),
 'letra': np.float64(8.200000000000001),
 'ne': np.float64(1.0)}

In [ ]:
con_oss2

{'abandonos': np.float64(8.9162),
 'botones': np.float64(61.513600000000004),
 'otros': np.float64(0.0),
 'texto': np.float64(7.0348),
 'x': np.float64(2.0450000000000004),
 'cambiar de tema': np.float64(2.3722000000000003)}

In [ ]:
cant_oss1

135127